In [1]:
# urn and other imports
import unityneuro.render as urn
import numpy as np
import pandas as pd
# ibl imports
from one.api import ONE
one = ONE(base_url='https://alyx.internationalbrainlab.org')
from brainbox.io.one import SpikeSortingLoader
import ibllib.atlas as atlas
ba = atlas.AllenAtlas(25)

C:\Users\Dan\.conda\envs\iblenv\lib\site-packages\one\api.py:1416: UserWarning: Newer cache tables require ONE version 1.13.0 or greater
  warnings.warn(f'Newer cache tables require ONE version {min_version} or greater')


In [108]:
def get_bwm_sessions():

    '''
    Gaelle, 06.04.2022
    '''

    str_query = (  
       'session__project__name__icontains,ibl_neuropixel_brainwide_01,'
       'session__json__IS_MOCK,False,session__qc__lt'
       ',50,~json__qc,CRITICAL,'
       'session__extended_qc__behavior,1,'
       'json__extended_qc__tracing_exists,True,'
       '~session__extended_qc___task_stimOn_goCue_delays__lt,0.9,'
       '~session__extended_qc___task_response_feedback_delays__lt,0.9,'
       '~session__extended_qc___task_wheel_move_before_feedback__lt,0.9,'
       '~session__extended_qc___task_wheel_freeze_during_quiescence__lt,0.9,'
       '~session__extended_qc___task_error_trial_event_sequence__lt,0.9,'
       '~session__extended_qc___task_correct_trial_event_sequence__lt,0.9,'
       '~session__extended_qc___task_reward_volumes__lt,0.9,'
       '~session__extended_qc___task_reward_volume_set__lt,0.9,'
       '~session__extended_qc___task_stimulus_move_before_goCue__lt,0.9,'
       '~session__extended_qc___task_audio_pre_trial__lt,0.9')                  
        
    str_query2 = (
       'session__project__name__icontains,ibl_neuropixel_brainwide_01,'
       'session__json__IS_MOCK,False,session__qc__lt,50,'
       '~json__qc,CRITICAL,session__extended_qc__behavior,1,'       
       'json__extended_qc__tracing_exists,True,'
       'session__extended_qc___experimenter_task,PASS')        
            
    ins = np.concatenate([one.alyx.rest('insertions', 'list', django = x)
                          for x in [str_query, str_query2]])

    eid_probe = set([x['session']+'_'+x['name'] for x in ins]) # pid via x['id']
    ins = [x.split('_') for x in eid_probe] 
 
    return ins

In [153]:
ins = get_bwm_sessions()

In [10]:
# def a function to get the pid for an eid/probe combo
def eid2pids(eid):
    insertions = one.alyx.rest('insertions', 'list', session=eid)
    pids = [i['id'] for i in insertions]
    return pids

# Collect insertion data (planned/micro-manip/histology)

In [23]:
def getCoords(ins):
    entry_coords = ba.xyz2ccf(ins.entry)
    tip_coords = ba.xyz2ccf(ins.tip)
    angles = [ins.phi, ins.theta, ins.beta]
    depth = np.sqrt(np.sum(np.power(entry_coords-tip_coords,2)))
    return (entry_coords, tip_coords, angles, depth)

def traj2coords(traj):
    insertion = atlas.Insertion.from_dict(traj)
    return getCoords(insertion)

In [167]:
traj_data = pd.DataFrame(columns=['pid','lab',
                                  'ml_e_p','ap_e_p','dv_e_p',
                                  'ml_t_p','ap_t_p','dv_t_p',
                                  'phi_p','theta_p','depth_p',
                                  'ml_e_m','ap_e_m','dv_e_m',
                                  'ml_t_m','ap_t_m','dv_t_m',
                                  'phi_m','theta_m','depth_m',
                                  'ml_e_e','ap_e_e','dv_e_e',
                                  'ml_t_e','ap_t_e','dv_t_e',
                                  'phi_e','theta_e','depth_e'])

In [140]:
prov_planned = 'planned'
prov_mm = 'Micro-manipulator'
prov_ephys = 'Ephys aligned histology track'

bwm_planned = one.alyx.rest('trajectories', 'list', provenance=prov_planned,
                          project='ibl_neuropixel_brainwide_01', use_cache=False)
bwm_mm = one.alyx.rest('trajectories', 'list', provenance=prov_mm,
                          project='ibl_neuropixel_brainwide_01', use_cache=False)
bwm_ephys = one.alyx.rest('trajectories', 'list', provenance=prov_ephys,
                          project='ibl_neuropixel_brainwide_01', use_cache=False)

In [151]:
bwm_planned

In [165]:
# re-organize data into a dictionary
data = {}
for (eid,probe) in ins:
    if not eid in data.keys():
        data[eid] = {}
    data[eid][probe] = {}
    
for traj in bwm_planned:
    eid = traj['session']['id']
    probe_name = traj['probe_name']
    
    if eid in data.keys() and probe_name in data[eid].keys():
        data[eid][probe_name]['planned'] = traj
        
for traj in bwm_mm:
    eid = traj['session']['id']
    probe_name = traj['probe_name']
    
    if eid in data.keys() and probe_name in data[eid].keys():
        data[eid][probe_name]['micro-manip'] = traj
        
for traj in bwm_ephys:
    eid = traj['session']['id']
    probe_name = traj['probe_name']
    
    if eid in data.keys() and probe_name in data[eid].keys():
        data[eid][probe_name]['ephys'] = traj

In [177]:
count = 0

for eid in data.keys():
    for probe_name in data[eid].keys():
        # print((eid,probe_name))
        
        
        tdata = data[eid][probe_name]
        missing = False
        
        if 'planned' in tdata.keys():
            (ecoords_p, tcoords_p, angles_p, depth_p) = traj2coords(tdata['planned'])
        else:
            print(f"{eid} {probe_name} missing planned")
            missing = True

        if 'micro-manip' in tdata.keys():
            (ecoords_m, tcoords_m, angles_m, depth_m) = traj2coords(tdata['micro-manip'])
        else:
            print(f"{eid} {probe_name} missing micro-manip")
            missing = True

        if 'ephys' in tdata.keys():
            (ecoords_e, tcoords_e, angles_e, depth_e) = traj2coords(tdata['ephys'])
        else:
            print(f"{eid} {probe_name} missing ephys")
            missing = True

        if missing:
            continue

        traj_data.loc[count] = [tdata['planned']['probe_insertion'],tdata['planned']['session']['lab'],
                            ecoords_p[0],ecoords_p[1],ecoords_p[2],
                            tcoords_p[0],tcoords_p[1],tcoords_p[2],
                            angles_p[0],angles_p[1],depth_p,
                            ecoords_m[0],ecoords_m[1],ecoords_m[2],
                            tcoords_m[0],tcoords_m[1],tcoords_m[2],
                            angles_m[0],angles_m[1],depth_m,
                            ecoords_e[0],ecoords_e[1],ecoords_e[2],
                            tcoords_e[0],tcoords_e[1],tcoords_e[2],
                            angles_e[0],angles_e[1],depth_e]
        count+=1

traj_data.to_csv('bwm_all_traj.csv',float_format='%.03f')

c557324b-b95d-414c-888f-6ee1329a2329 probe01 missing planned
e45481fa-be22-4365-972c-e7404ed8ab5a probe00 missing ephys
edd22318-216c-44ff-bc24-49ce8be78374 probe01 missing planned
35ed605c-1a1a-47b1-86ff-2b56144f55af probe01 missing planned
35ed605c-1a1a-47b1-86ff-2b56144f55af probe00 missing planned
d62a64f4-fdc6-448b-8f2a-53ed08d645a7 probe00 missing ephys
20c112a1-8a42-44e0-a4cd-e5b932f7bda9 probe01 missing planned
20c112a1-8a42-44e0-a4cd-e5b932f7bda9 probe00 missing planned
8c552ddc-813e-4035-81cc-3971b57efe65 probe00 missing ephys
8c552ddc-813e-4035-81cc-3971b57efe65 probe01 missing ephys
5139ce2c-7d52-44bf-8129-692d61dd6403 probe00 missing planned
aa3432cd-62bd-40bc-bc1c-a12d53bcbdcf probe00 missing ephys
a9138924-4395-4981-83d1-530f6ff7c8fc probe00 missing planned
a9138924-4395-4981-83d1-530f6ff7c8fc probe01 missing planned
f99ac31f-171b-4208-a55d-5644c0ad51c3 probe00 missing planned
695a6073-eae0-49e0-bb0f-e9e57a9275b9 probe01 missing planned
c7e4e6ad-280f-432f-ac85-9be299890d

In [178]:
traj_data

,pid,lab,ml_e_p,ap_e_p,dv_e_p,ml_t_p,ap_t_p,dv_t_p,phi_p,theta_p,...,depth_m,ml_e_e,ap_e_e,dv_e_e,ml_t_e,ap_t_e,dv_t_e,phi_e,theta_e,depth_e
0,e8129a86-b5a9-4d2e-9e9c-09689c9bf0b3,cortexlab,4435.0,7650.0,264.0,3285.275416,7650.000000,6784.412133,0.0,10.0,...,6348.000,4471.334154,7498.130724,225.0,3206.935707,7586.750803,6931.193545,4.009232,10.702902,6824.924089
1,adca9242-0725-41d4-8eb1-5655a464431d,cortexlab,4939.0,4400.0,954.0,5974.276180,4400.000000,4817.703305,180.0,15.0,...,4001.000,4900.000000,4150.000000,1075.0,5535.037085,5073.505888,5252.945974,124.513864,15.016596,4325.663855
2,d4291925-ad00-47fe-baaf-3fdff0991e86,cortexlab,5039.0,8975.0,332.0,5815.457135,8975.000000,3229.777479,180.0,15.0,...,4000.000,5075.000000,8775.000000,275.0,5719.955621,8458.254095,4038.003837,-153.843758,10.810369,3830.991203
3,4ea45238-55b1-4d54-ba92-efa47feb9f57,cortexlab,3039.0,9900.0,332.0,1778.340982,10010.293373,3808.862697,5.0,20.0,...,3900.000,3175.000000,11200.000000,1475.0,2355.793444,11018.539115,5675.101794,-12.489811,11.297380,4283.092635
4,c6ba6f8e-c13e-410f-b7df-e193ba0d239d,danlab,3471.0,9900.0,692.0,4506.276180,9900.000000,4555.703305,180.0,15.0,...,4000.000,3300.000000,9750.000000,825.0,4018.222006,9886.012241,4584.740640,169.276680,11.002461,3830.142537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,2e720cee-05cc-440e-a24b-13794b1ac01d,churchlandlab_ucla,3279.0,9175.0,332.0,4320.889066,9175.000000,6240.846518,180.0,10.0,...,5999.276,2950.000000,9250.000000,775.0,5256.838358,11711.594108,6311.341791,133.141163,31.356013,6483.211333
518,8dfb86c8-d45c-46c4-90ec-33078014d434,angelakilab,4935.0,6150.0,332.0,3827.124626,6150.000000,6615.073464,0.0,10.0,...,6380.000,4675.000000,5925.000000,525.0,4401.221324,7186.911353,6718.068479,77.759068,11.777565,6326.252620
519,94e948c1-f7be-4868-893a-f7cd2df3313e,churchlandlab_ucla,3496.0,7400.0,501.0,4531.276180,7400.000000,4364.703305,180.0,15.0,...,0.000,3475.000000,8350.000000,425.0,4478.592268,8354.150996,5236.127350,179.763018,11.782928,4914.688297
520,eb99c2c8-e614-4240-931b-169bed23e9f5,danlab,3496.0,6900.0,549.0,4531.276180,6900.000000,4412.703305,180.0,15.0,...,4000.000,2950.000000,7525.000000,625.0,3417.638831,7622.773402,4433.161139,168.190777,7.150651,3838.012112


In [ ]:
traj_data = pd.read_csv('bwm_all_traj.csv')

# Discrepancy 1: Planned identical to micro-manipulator
Analyze the *surface* coordinate and check whether it is identical from planned to micro-manipulator, suggesting the researcher did not enter the real micro-manipulator coordinates

In [180]:
def disc1(row):
    planned = (row['ml_e_p'], row['ap_e_p'], row['dv_e_p'])
    micro = (row['ml_e_m'], row['ap_e_m'], row['dv_e_m'])
    return planned[0]==micro[0] and planned[1]==micro[1] and planned[2]==micro[2]

In [189]:
p_identical_mm = []
for i, row in traj_data.iterrows():
    p_identical_mm.append(disc1(row))

traj_data['p_identical_mm'] = p_identical_mm

traj_data.to_csv('bwm_all_traj.csv',float_format='%.03f')

In [190]:
print(np.sum(traj_data['p_identical_mm']))

79


# Discrepancy 2: >1mm distance from micro-manipulator to ephys
Analyze the surface coordinate to check whether the micro-manipulator coordinate is >1mm away from the ephys coordinate
we'll use only ap/ml and ignore dv for this

In [198]:
def disc2(row):
    micro = (row['ml_e_m'], row['ap_e_m'], row['dv_e_m'])
    ephys = (row['ml_e_e'], row['ap_e_e'], row['dv_e_e'])
    return np.hypot(micro[0]-ephys[0],micro[1]-ephys[1]) > 1000

In [200]:
mm_ephys_dist = []
for i, row in traj_data.iterrows():
    mm_ephys_dist.append(disc2(row))

traj_data['mm_ephys_dist'] = mm_ephys_dist

traj_data.to_csv('bwm_all_traj.csv',float_format='%.03f')

In [201]:
print(np.sum(traj_data['mm_ephys_dist']))

23


# Discrepancy 3: Flipped DV axis
Check whether the DV coordinate for micro-manipulator and ephys are flipped relative to zero (suggesting the researcher doesn't know which way is which)

In [213]:
def disc3(row):
    micro = (row['ml_e_m'], row['ap_e_m'], row['dv_e_m'])
    ephys = (row['ml_e_e'], row['ap_e_e'], row['dv_e_e'])
    return (micro[2] > 0 and ephys[2] < 0) or (micro[2] < 0 and ephys[2] > 0)

In [214]:
flipz = []
for i, row in traj_data.iterrows():
    flipz.append(disc3(row))

traj_data['flipz'] = flipz

traj_data.to_csv('bwm_all_traj.csv',float_format='%.03f')

In [216]:
print(np.sum(traj_data['flipz']))

55


# Discrepancy 4: 0 in micro-manipulator DV position

In [218]:
def disc4(row):
    micro = (row['ml_e_m'], row['ap_e_m'], row['dv_e_m'])
    return micro[2]==0

In [219]:
mm_zero_dv = []
for i, row in traj_data.iterrows():
    mm_zero_dv.append(disc4(row))

traj_data['mm_zero_dv'] = mm_zero_dv

traj_data.to_csv('bwm_all_traj.csv',float_format='%.03f')

In [220]:
print(np.sum(traj_data['mm_zero_dv']))

0


# Discrepancy 5: crazy angles

In [221]:
def disc5(row):
    phi_m = row['phi_m']
    phi_e = row['phi_e']
    return np.abs(phi_m-phi_e) > 45

In [222]:
big_phi_em = []
for i, row in traj_data.iterrows():
    big_phi_em.append(disc5(row))

traj_data['big_phi_em'] = big_phi_em

traj_data.to_csv('bwm_all_traj.csv',float_format='%.03f')

In [223]:
print(np.sum(traj_data['big_phi_em']))

142


# Display data using Urchin

In [11]:
import random
r = lambda: random.randint(0,255)
randHexColor = lambda : '#%02X%02X%02X' % (r(),r(),r())

In [22]:
colors = {}
labs = np.unique(traj_data['lab'].values)

for lab in labs:
    colors[lab] = randHexColor()

In [9]:
urn.setup()

(URN) connected to server
Login sent with ID: Dan


In [ ]:
# VIEW 1: Planned insertions
urn.clear()
count = 0

for i, row in traj_data.iterrows():
    probename = 'p'+str(i)
    urn.create_probes([probename])
    urn.set_probe_positions({probename:[row.ml, row.ap, row.dv]})
    urn.set_probe_colors({probename:'#808080'})
    urn.set_probe_angles({probename:[row.phi, row.theta, 0]})
    urn.set_probe_size({probename:[0.07,3.84,0.02]})

# # for some reason 'root' doesn't work?
urn.set_area_visibility({8:True})
urn.set_area_material({8:'transparent-unlit'})
urn.set_area_color({8:'#000000'})
urn.set_area_alpha({8:0.025})

In [56]:
urn.clear()